In [1]:
from fastcore.all import *
from fastprogress import progress_bar
import random, json

random.seed(0)

In [2]:
%cd /storage/Data/Yelp/photos

/storage/Data/Yelp/photos


In [3]:
photos = []
with open('../photos.json') as f:
    for line in f:
        photos.append(json.loads(line))
photos = L(photos)

In [4]:
photos[0]

{'photo_id': '001kKc6P_rmgaXThAHxEkw',
 'business_id': 'zKZc8GwCaKbACaV0HZAmpg',
 'caption': 'Clean and simple interior',
 'label': 'inside'}

In [5]:
id2entry = {photo['photo_id']: photo for photo in photos}

In [6]:
all_photos = Path('.').ls()

In [7]:
all_photos.shuffle();

In [8]:
for i, batch in enumerate(progress_bar(list(chunked(all_photos, chunk_sz=5_000, drop_last=False)))):
    filenames = '\0'.join(L(batch).attrgot('name'))
    batch_dir = Path(f'../batches/{i:03d}')
    batch_dir.mkdir(exist_ok=True)
    source_file = batch_dir / 'files.txt'
    Path(source_file).write_text(filenames, encoding = 'utf-8')
    run(['tar', 'cf', batch_dir / 'photo_batch.tar', '--verbatim-files-from', '--files-from', source_file])
    json_data = [id2entry[o.name[:-4]] for o in batch]
    (batch_dir / 'photos.json').write_text('\n'.join(json.dumps(p) for p in json_data))
